In [ ]:
import h2o
from h2o.estimators import H2ODeepLearningEstimator
import pandas as pd

In [ ]:
h2o.init()

In [ ]:
#loop through all results
for i in range(1,24):
    #read in the training and test data
    print("i: %s" % (i))
    train_file1 = "../data/train/train%s.csv" % (i)
    #print(train_file1)
    train1 = h2o.import_file(train_file1)
    train1["Treatment.Group"] = train1["Treatment.Group"].asfactor()

    
    
    test_file1 = "../data/test/test%s.csv" % (i)
    #print(test_file1)
    test1 = h2o.import_file(test_file1)
    test1["Treatment.Group"] = test1["Treatment.Group"].asfactor()
    testing_file = pd.read_csv(test_file1)
    
    #specify the columns to use for training and the variable to predict
    x = train1.columns
    y = "DELTA_AREA"
    x.remove(y)
    
    #initilize the model
    dl = H2ODeepLearningEstimator(seed=428)
    #train the model
    dl.train(x=x,y=y,training_frame=train1)
    #predict the change in area using the model on the test sample 
    predicted_values = dl.predict(test1)
    #save output
    export_path = "../output/actual_vs_predicted/predicted_value%s" % (i)
    export_path = export_path + ".csv"
    h2o.export_file(predicted_values, path = export_path, force = True)
    actual = pd.DataFrame(testing_file["DELTA_AREA"])
    export_actual = "../output/actual_vs_predicted/actual_value%s" % (i)
    actual.to_csv(export_actual,index=False)

In [ ]:
#get features of importance using all samples as training and test
train1 = h2o.import_file("../data/delta_area_for_regression.csv")
x = train1.columns
print(x)
y = "DELTA_AREA"
train1["\ufeffTreatment Group"] = train1["\ufeffTreatment Group"].asfactor()

x.remove(y)
dl = H2ODeepLearningEstimator(seed=428)
dl.train(x=x,y=y,training_frame=train1)
predicted_values = dl.predict(train1)
#get features of importance and save
varimp = dl.varimp(use_pandas=True)
varimp.to_csv("../output/features_of_importance_Delta_Area.csv",index=False)